# Introdução

Nosso projeto consiste em avaliar o processo de visto americano para trabalho, o H1B, e se dependendo de certos fatores uma pessa receberia o visto ou não. Para tirar este visto é necessário passar informações como, o nome do lugar de trabalho, com o que a pessoa trabalhará, se é um trabalho de tempo integral, o salário, e a cidade onde será o trabalho (traduzido para latitude e longitude), todos dados contidos no nosso dataset. Porém havia um problema nosso dataset, obtido no Kaggle havia 3 milhões de linhas, um arquivo muito grande, fazendo com que fique difícil e demorado para se trabalhar. Por esse motivo cortamos o dataset, no qual o processo esta em um outro notebook chamado Shuffle dataset.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
#abrindo o dataframe
df = pd.read_csv('Dataframe.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,1805963,1,"SAGARSOFT, INC",Computer and Information Systems Managers,PROJECT MANAGER,Y,69222.4,2013.0,"SHELTON, CONNECTICUT",-73.093164,41.316486
1,1,767976,0,MOUNT AUBURN HOSPITAL,PHYSICAL THERAPISTS,PHYSICAL THERAPIST,Y,89648.0,2015.0,"CAMBRIDGE, MASSACHUSETTS",-71.109733,42.373616
2,2,376800,1,"ERP ANALYSTS, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",SOFTWARE PROGRAMMER/ANALYST,Y,78957.0,2016.0,"CREVE COEUR, MISSOURI",-90.422618,38.660885
3,3,1233483,1,"ORCHESTRA TECHNOLOGY, INC",NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,IMS CORE QA TEST ENGINEER,Y,63190.0,2015.0,"BELLEVUE, WASHINGTON",-122.201516,47.610150
4,4,2235324,0,"SAP INDUSTRIES, INC.",Computer Systems Analysts,APPLICATIONS CONSULTANT IV,Y,97635.0,2012.0,"IRVING, TEXAS",-96.948894,32.814018


Na célula abaixo, o tratamento dos dados é feito. Em primeiro lugar as colunas Unnamed: 0 e Unnamed: 0.1, são retiradas, essas correspondem ao index atual e ao index antes do embaralhamento feito no outro notebook. O local de trabalho, WORKSITE, também é retirado uma vez que esse já está no dataset em forma de Latitude e Longitude, variáves quantitavas. Essas então são transformadas para valores de 0 a 1 para facilitar a forma de se trabalhar, bem como a variável de trabalho em tempo intergral, FULL_TIME_POSITION, também é transformada, de N ou Y, para 0 ou 1. JOB_TITLE é retirado pois a variável SOC_NAME, do Inglês Standard Occupational Classification (SOC), já corresponde ao trabalho ocupado porém em uma forma de classificação padrão e essa então é transormada para uma váriavel quantitativa através do metodo de One-hot Encoding. Por fim o ano é retirado uma vez que queremos prever o visto de anos futuros então não se pode levar em conta os anos passados.  

In [3]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def mapp (value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


df = df.drop(["WORKSITE","YEAR","JOB_TITLE","Unnamed: 0","Unnamed: 0.1"], axis=1)
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1)

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)



latmin = df["lat"].min()
latmax = df["lat"].max()
lonmin = df["lon"].min()
lonmax = df["lon"].max()

df["lat"] = df["lat"].apply(lambda x: mapp(x, latmin, latmax, 0, 1))
df["lon"] = df["lon"].apply(lambda x: mapp(x, lonmin, lonmax, 0, 1))

df = dummify(df, "SOC_NAME")

In [4]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,SOC_NAME_ACTUARIES,...,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, SECONDARY SCHOOL",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,","SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS","SOC_NAME_WOODWORKERS, ALL OTHER",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,69222.4,0.279211,0.542393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,89648.0,0.285744,0.562959,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,78957.0,0.222129,0.490728,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,63190.0,0.117451,0.664836,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,97635.0,0.200632,0.376977,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O X e y do modelo, baseando-se no CASE_STATUS(se foi aprovado ou não) são separados em base para treinamentos e base para teste, sendo a divisão feita de 67 e 33 por cento. Em seguida é aplicado o modelo, neste caso de regressão logística, uma vez que esse método consegue fazer a predição de para valores a partir das nossas variáves que foram transformadas em quantitativas.

In [5]:
X = df.drop(["CASE_STATUS"], axis=1).values
y = df["CASE_STATUS"].values

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=RANDOM_SEED)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((67000, 811), (67000,), (33000, 811), (33000,))

Na celula abaixo, faz-se a implementação do classificador RandomForest. Este modelo foi escolhido por nós pois, ao se analisar a chamada 'flow chart' do Sklearn, a categpria que mais se alinhava com nosso modelo era a "ensemble classifier". O classificador RandomForest faz parte desta categoria. 

Após implementarmos o modelo, calculamos sua precisão. Para fazermos isso, separamos o dataframe em duas amostras de 50000: a primeira para treinar o classificador e a segunda para testá-lo de fato. Depois disso, comparamos os resultados obtidos com os reais, sendo que nosso classificador 'acertava' em 65% dos casos. 

In [6]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Precisão: {}%".format(100*accuracy_score(y_test, y_pred)))

C:\Users\Freddy Dratwa\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Precisão: 65.33939393939394%


In [7]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,SOC_NAME_ACTUARIES,...,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, SECONDARY SCHOOL",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,","SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS","SOC_NAME_WOODWORKERS, ALL OTHER",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,69222.4,0.279211,0.542393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,89648.0,0.285744,0.562959,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,78957.0,0.222129,0.490728,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,63190.0,0.117451,0.664836,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,97635.0,0.200632,0.376977,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Na celula abaixo, com intuito de melhorar a análise da eficiência do modelo, adicionamos a matriz de confusão que compara os valores reais com os previstos pelo modelo. Como podemos ver, 11163 vezes o previsto foi 0 e o real também, e assim por diante. 

In [8]:
pd.crosstab(y_test, model.predict(X_test),rownames=['Real'],colnames =['Previsto'],margins=True)

Previsto,0,1,All
Real,,,
0,11163,5403,16566
1,6035,10399,16434
All,17198,15802,33000


Na célula abaixo, um usuário poderia passar suas infromações ao classificador e a partir delas, ele responderia se ela seria ou nao aprovada à receber o visto. Vale ressaltar que o classificador acerta em 65% dos casos, ou seja, a resposta dada por ele ainda tem 35% de chance de estar errada. 

In [19]:
def perguntas():
    print("Olá, responda essas perguntas para saber se uma pessoa teria o visto americano de trbalho H1B.")
    f = int(input("A pessoa tem trabalho integral?(0-Não/1-Sim) "))
    w = float(input("Qual o salário da pessoa? USD$/Ano "))
    lo = float(input("Qual a longitude da cidade(apenas do EUA e em número inteiro)? "))
    la = float(input("Qual a latitude da cidade(apenas do EUA e em número inteiro)? "))
    return f, w, lo, la

In [20]:
#responde, com base no modelo se uma pessoa com as informações fornecidas seria aprovada ou não.

trabalhos = df.columns.values
trabalhos = trabalhos[5:]
temp = []
for i in trabalhos:
    temp.append(i[9:])
    


try:
    fulltime, wage, longi, lati = perguntas()
    
    longi = mapp(longi, lonmin, lonmax, 0, 1)
    lati = mapp(lati, latmin, latmax, 0, 1)

    
    job = "SOC_NAME_"
    d = {"FULL_TIME_POSITION":fulltime, "PREVAILING_WAGE":wage, "lon":longi, "lat":lati, job:1}
    l=[]

    for i in df.columns:
        if i != "CASE_STATUS":
            if i in d:
                l.append(d[i])
            else:
                l.append(0)

    # print(l)
    resultado = model.predict([l])
    # print(resultado)

    if resultado == [1]:
        print("Essa pessoa teria sido aprovada")
    else:
        print("Essa pessoa teria sido reprovada")

except:
    print("Erro em algum valor passado. ")


Olá, responda essas perguntas para saber se uma pessoa teria o visto americano de trbalho H1B.
A pessoa tem trabalho integral?(0-Não/1-Sim) 1
Qual o salário da pessoa? USD$/Ano 10000
Qual a longitude da cidade(apenas do EUA e em número inteiro)? 11
Qual a latitude da cidade(apenas do EUA e em número inteiro)? 11
Essa pessoa teria sido reprovada


## Conclusão

A partir dos resultados obtidos, percebe-se que o modelo tem uma precisão de quase 65%. Desta forma, conclui-se que o modelo não é tão preciso. Portanto, se uma pessoa deseja saber se terá seu visto para os EUA concedido, basta que ela use o modelo com suas informações. A resposta obtida terá 65% de probabilidade de condizer com a verdadeira.